In [ ]:
# ============================================================
# 📘 Pegos - Offline Data Generator (for GitHub Actions)
# ============================================================
# Bu notebook internete bağlanmadan, örnek veri oluşturur
# ve "data/output.csv" dosyasına kaydeder.
# ============================================================

import pandas as pd
import datetime
import random
import os

In [ ]:
# ============================================================
# 1️⃣ Klasör oluştur
# ============================================================
os.makedirs("data", exist_ok=True)

In [ ]:
# ============================================================
# 2️⃣ Sahte veriler oluştur (örnek amaçlı)
# ============================================================
timestamps = [datetime.datetime.now().isoformat()]
btc_price = round(random.uniform(60000, 70000), 2)
eth_price = round(btc_price * random.uniform(0.05, 0.07), 2)

data = {
    "timestamp": timestamps,
    "btc_price_usd": [btc_price],
    "eth_price_usd": [eth_price],
    "source": ["simulated-offline"]
}

df = pd.DataFrame(data)

In [ ]:
# ============================================================
# 3️⃣ Ek bilgiler ekle
# ============================================================
df["date"] = pd.to_datetime(df["timestamp"]).dt.date
df["time"] = pd.to_datetime(df["timestamp"]).dt.time


In [ ]:
# ============================================================
# 4️⃣ Veriyi kaydet
# ============================================================
output_path = "data/output.csv"
df.to_csv(output_path, index=False)

print(f"✅ Veri oluşturuldu ve kaydedildi: {output_path}")
print(df)

In [ ]:
# ============================================================
# 5️⃣ Log dosyasına yaz
# ============================================================
log_entry = f"[{datetime.datetime.now()}] Data saved: {output_path}\n"
with open("data/fetch_log.txt", "a") as log_file:
    log_file.write(log_entry)
print("🗒️ Log dosyasına kayıt eklendi.")
